In [15]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

In [16]:
def extract_permission_values(xml_file):
    # Check if the file exists
    if not os.path.exists(xml_file):
        print(f"Error: File '{xml_file}' does not exist.")
        return ["Not Found"]

    # Parse the XML file
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
    except ET.ParseError:
        print(f"Error: Unable to parse XML file '{xml_file}'")
        return []

    # Namespace used in the XML (for ns0:name)
    namespace = {'ns0': 'http://schemas.android.com/apk/res/android'}

    # Extract permissions for both cases: ns0:name and name
    permissions = []
    
    # Case 1: Extract ns0:name attributes (namespaced)
    permissions += [perm.attrib['{http://schemas.android.com/apk/res/android}name'] 
                    for perm in root.findall('uses-permission', namespace) 
                    if '{http://schemas.android.com/apk/res/android}name' in perm.attrib]
    
    # Case 2: Extract name attributes (non-namespaced)
    permissions += [perm.attrib['name'] 
                    for perm in root.findall('uses-permission') 
                    if 'name' in perm.attrib]

    return permissions
def read_csv_and_filter(file_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Filter rows where the 'manifest' column has a NaN value
    filtered_df = df[df['manifest_permission'].isna()]

    # Extract the 'apkfile_name' column from the filtered DataFrame
    apkfile_names = filtered_df['apkfile_name_new'].tolist()
    pkg_names = filtered_df['pkg_name'].tolist()
    return apkfile_names, pkg_names
def array_to_string(array):
    return '\n'.join(array)
# Funtion update CSV data
# def update_csv_column(csv_path, condition_column, condition_value, update_column, new_value):
#     # Read the CSV file into a DataFrame
#     df = pd.read_csv(csv_path)
    
#     # Ensure the update_column is of type object (can hold strings)
#     df[update_column] = df[update_column].astype(object)

#     # Update the specified column based on the condition
#     df.loc[df[condition_column] == condition_value, update_column] = str(new_value)

#     # Write the updated DataFrame back to the CSV file
#     df.to_csv(csv_path, index=False)
#     print(f"CSV updated at {csv_path}")
def update_csv_column(csv_path, condition_column, condition_value, update_column, new_value):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)
    
    # Ensure the update_column is of type object (can hold strings)
    df[update_column] = df[update_column].astype(object)

    # Print the matching rows (for debugging purposes)
    print(f"Checking for rows where {condition_column} == {condition_value}")
    matching_rows = df.loc[df[condition_column] == condition_value]
    print(f"Found {len(matching_rows)} matching rows")

    # Update the specified column based on the condition
    if len(matching_rows) > 0:
        df.loc[df[condition_column] == condition_value, update_column] = str(new_value)
        print(f"Updating {update_column} for {condition_value} with {new_value}")
    else:
        print(f"No matching rows found for {condition_value} in {condition_column}")
    
    # Write the updated DataFrame back to the CSV file
    df.to_csv(csv_path, index=False)
    print(f"CSV updated at {csv_path}")

In [17]:
# 1. Read CSV data file
# csv_path = "C:\\Users\\ASUS\\anaconda3\\wearable-apk-manifest\\combine_filtered_cleaned.csv"
# apk_path = "C:\\Users\\ASUS\\anaconda3\\wearable-standard-app\\apk"
# xml_directory = ".\\manifest-wearable"

# stand-alone
csv_path = "C:\\Users\\ASUS\\anaconda3\\wearable-apk-manifest\\apk-wearable-standalone.csv"
#apk_path = "C:\\Users\\ASUS\\anaconda3\\wearable-standard-app\\apk"
xml_directory = r"C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone"

In [18]:
# MAIN
apk_file_name_arr, pkg_name_arr  = read_csv_and_filter(csv_path)
print((apk_file_name_arr))
print(pkg_name_arr)

['app.groupcal.www', 'apps.r.compass', 'ch.publisheria.bring', 'com.albuquerquedesign.adanalog013', 'com.anghami', 'com.audible.application', 'com.c25k', 'com.cardiogram.v1', 'com.centr.app', 'com.codverter.wearflashlight', 'com.contorra.golfpad', 'com.customsolutions.android.alexa', 'com.dungelin.heartrate', 'com.exovoid.weather.app', 'com.fish4fun.mycards', 'com.fitbod.fitbod', 'com.fitiv.fitivapplication', 'com.fiton.android', 'com.flightradar24free', 'com.fourtechnologies.mynetdiary.ad', 'com.fsoydan.howistheweather', 'com.funnmedia.waterminder', 'com.furyapps.scoreswidget', 'com.GD.Minimal', 'com.golfbuddy.smartcaddie', 'com.google.android.apps.maps', 'com.google.android.apps.youtube.music', 'com.google.android.contacts', 'com.google.android.deskclock', 'com.google.android.keep', 'com.grint.thegrint.pro', 'com.hrd.iam', 'com.hrd.motivation', 'com.imperon.android.gymapp', 'com.ingravity.woo', 'com.jee.calc', 'com.JiaRen.LCDsimple', 'com.juventus.app.android', 'com.krisdb.wearcasts'

In [19]:
for i in range(len(apk_file_name_arr)):
    print("---------------------------- Loop-"+str(i)+" ----------------------------")
    apkfile_name_new = apk_file_name_arr[i]
    pkg_name = pkg_name_arr[i]
    print("apkfile_name_new:",apkfile_name_new)
    print("pkg_name:",pkg_name)
#     manifest_path = xml_directory+"\\"+str(os.path.splitext(apkfile_name_new)[0])+"-AndroidManifest.xml" 
    ### For standalone
    manifest_path = xml_directory+"\\"+pkg_name+"-AndroidManifest.xml"
    print("manifest_path",manifest_path)
    permission_arr = extract_permission_values(manifest_path)
    print("permission_arr",permission_arr)
    print("len permission arr",len(permission_arr))
    if (len(permission_arr)>0 and permission_arr[0]!="Manifest Not Found"):
        permission_str = array_to_string(permission_arr)
        print("permission_str",permission_str)
        update_csv_column(csv_path, "pkg_name", pkg_name, "manifest_permission", permission_str)
    elif (len(permission_arr)>0 and permission_arr[0]=="Not Found"):
        print("Manifest file not found")
        update_csv_column(csv_path, "pkg_name", pkg_name, "manifest_permission", "Not Found")
    elif (len(permission_arr)==0):
        print("No have permission")
        update_csv_column(csv_path, "pkg_name", pkg_name, "manifest_permission", "No Permission")
    else:
        print("Unknow")
    #break

---------------------------- Loop-0 ----------------------------
apkfile_name_new: app.groupcal.www
pkg_name: app.groupcal.www
manifest_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\app.groupcal.www-AndroidManifest.xml
permission_arr ['android.permission.READ_CONTACTS', 'android.permission.WRITE_CALENDAR', 'android.permission.READ_CALENDAR', 'android.permission.READ_SYNC_SETTINGS', 'android.permission.READ_SYNC_STATS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.INTERNET', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.USE_EXACT_ALARM', 'android.permission.FOREGROUND_SERVICE', 'android.permission.VIBRATE', 'android.permission.SCHEDULE_EXACT_ALARM', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.POST_NOTIFICATIONS', 'com.android.vending.BILLING', 'android.permission.WAK

CSV updated at C:\Users\ASUS\anaconda3\wearable-apk-manifest\apk-wearable-standalone.csv
---------------------------- Loop-30 ----------------------------
apkfile_name_new: com.grint.thegrint.pro
pkg_name: com.grint.thegrint.pro
manifest_path C:\Users\ASUS\anaconda3\wearable-apk-manifest\manifest-wearable-standalone\com.grint.thegrint.pro-AndroidManifest.xml
permission_arr ['android.permission.INTERNET', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.READ_MEDIA_VIDEO', 'android.permission.READ_MEDIA_IMAGES', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.ACCESS_WIFI_STATE', 'android.permission.VIBRATE', 'com.android.vending.CHECK_LICENSE', 'android.permission.READ_PHONE_STATE', 'com.google.android.providers.gsf.permission.READ_GSERVICES', 'com.grint.thegrint.pro.permission.MAPS_RECEIVE', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.CAMERA', 'andro

permission_arr ['android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.ACCESS_BACKGROUND_LOCATION', 'android.permission.POST_NOTIFICATIONS', 'android.permission.SCHEDULE_EXACT_ALARM', 'android.permission.ACCESS_NETWORK_STATE', 'android.permission.FOREGROUND_SERVICE', 'android.permission.INTERNET', 'android.permission.RECORD_AUDIO', 'android.permission.VIBRATE', 'android.permission.WAKE_LOCK', 'com.android.vending.BILLING', 'android.permission.RECEIVE_BOOT_COMPLETED', 'com.todoist.DYNAMIC_RECEIVER_NOT_EXPORTED_PERMISSION', 'com.google.android.c2dm.permission.RECEIVE', 'com.google.android.providers.gsf.permission.READ_GSERVICES', 'android.permission.ACCESS_ADSERVICES_ATTRIBUTION', 'android.permission.ACCESS_ADSERVICES_AD_ID', 'com.google.android.finsky.permission.BIND_GET_INSTALL_REFERRER_SERVICE']
len permission arr 19
permission_str android.permission.ACCESS_FINE_LOCATION
android.permission.ACCESS_COARSE_LOCATION
android.permission.A